# Generative AI App
Loads documents into a vector database (chroma) and  uses GPT4All's Mistral 7b Open Orca LLM.

In [1]:
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import GPT4AllEmbeddings
from langchain.vectorstores import Chroma
import sys

In [2]:
# Load Model
from langchain.llms import GPT4All

llm = GPT4All(
    model="/home/phisinger/Programmieren/wahlprogramm_analyse/models/mistral-7b-openorca.Q4_0.gguf",
    max_tokens=2048,
)

In [3]:
sys.path.insert(1, '/home/phisinger/Programmieren/wahlprogramm_analyse')
from GenAI.Vectorstore import Vectorstore_client

db_client = Vectorstore_client()
db_client = db_client.get_client()

langchain_chroma = Chroma(
    client=db_client,
    collection_name="2021",
    embedding_function=GPT4AllEmbeddings(),
)

# Prepare chain to ask questions
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

german_prompt = """Beantworten Sie die Frage am Ende des Textes anhand der folgenden Informationen. Wenn Sie die Antwort nicht wissen, sagen Sie einfach, dass Sie es nicht wissen, versuchen Sie nicht, eine Antwort zu erfinden.

{context}

Frage: {question}
Hilfreiche Antwort:"""
german_prompt = PromptTemplate(
    template=german_prompt, input_variables=["context", "question"]
)


qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=langchain_chroma.as_retriever(),
    chain_type_kwargs={"prompt": german_prompt},
)

bert_load_from_file: gguf version     = 2
bert_load_from_file: gguf alignment   = 32
bert_load_from_file: gguf data offset = 695552
bert_load_from_file: model name           = BERT
bert_load_from_file: model architecture   = bert
bert_load_from_file: model file type      = 1
bert_load_from_file: bert tokenizer vocab = 30522


In [4]:
# Ask a question
question = "Wie wollen die Grünen die Bahn umbauen?"
qa_chain({"query": question})

{'query': 'Wie wollen die Grünen die Bahn umbauen?',
 'result': ' Die Grünen möchten die Bahn umfassend ausbauen und für alle bezahlbar machen, indem sie das Angebot und den Streckennetz der Bahn flächendeckend erweitern. Sie fordern verlässliche, getaktete Fahrpläne und barrierefreie und nutzerfreundliche Bahnhöfe und Züge mit Servicepersonal. Darüber hinaus wollen sie die Geschäftspolitik der Deutschen Bahn am gemeinwohlertrag und der ökologischen Nachhaltigkeit ausrichten, um eine Beschleunigung in der Umsetzung zu erreichen. Sie planen auch eine umfassende Ausbildungs- und Forschungsoffensive einzuleiten, um die Leistungsfähigkeit und Zuverlässigkeit der Bahn weiter zu verbessern. Zudem wollen sie den Deutschlandtakt weiterentwickeln und realisieren, um attraktive und für alle bezahlbare Mobilitätsangebote in Stadt und Land zu schaffen.\n'}

: 